# **Phenological diversity with Sentinel 2**

### <span style="color:red">   Import packages</span>

In [1]:
import ee
ee.Initialize()
import geemap
Map = geemap.Map()

In [2]:
import pandas as pd

### <span style="color:red">  Load collections</span>

In [3]:
# forest mask from ESA CCI (unchanged forests)
forestmask = ee.Image('users/marcogirardello/phenoutils/mask_unchanged_500m')
# entire collection from Sentinel 2
collectionS2 = ee.ImageCollection("COPERNICUS/S2_SR")
# sampling areas
samples = ee.FeatureCollection('users/marcogirardello/exploratoryproject/samples')

### <span style="color:red">   Required functions</span>

In [117]:
def maskCloudAndShadows(image):
    cloudProb = image.select('MSK_CLDPRB')
    snowProb = image.select('MSK_SNWPRB')
    cloud = cloudProb.lt(5)
    snow = snowProb.lt(5)
    scl = image.select('SCL')
    shadow = scl.eq(3) # 3 = cloud shadow
    cirrus = scl.eq(10) # 10 = cirrus
    # cloud probability less than 5% or cloud shadow classification
    mask = (cloud.And(snow)).And(cirrus.neq(1)).And(shadow.neq(1))
    return image.updateMask(mask) 

# add dn for product aggregated into 16 days
def add_dn_date(img,beginDate=None,n=None,IncludeYear=False):
    if beginDate is None:
        beginDate = img.get('system:time_start')
    else:
        beginDate = beginDate
    if IncludeYear is False:
        IncludeYear = True
    if n is None:
        n = 8
    beginDate = ee.Date(beginDate)
    year  = beginDate.get('year')
    month = beginDate.get('month')
    diff  = beginDate.difference(ee.Date.fromYMD(year, 1, 1), 'day').add(1)
    dn    = diff.subtract(1).divide(n).floor().add(1).int()
    yearstr  = year.format('%d') 
    dn = dn.format('%02d')
    return ee.Image(img).set('system:time_start', beginDate.millis()).set('date', beginDate.format('yyyy-MM-dd')).set('Year', yearstr).set('Month',beginDate.format('MM')).set('YearMonth', beginDate.format('YYYY-MM')).set('dn', dn)

# wrapper function for add_dn_date
def add_dn_date_all(Year,days):
    def wrapper(image0):
        tmp = add_dn_date(img = image0,IncludeYear=Year,n = days)
        return tmp
    return (wrapper)

def aggregate(month):
    month = ee.String(month)
    seqNDVI = collection.filterMetadata('dn', 'equals',month)
    return seqNDVI.median().copyProperties(seqNDVI.first(), ['system:time_start','system:time_end','dn','Month'])


### <span style="color:red">  Filter random square</span>

In [5]:
onesq = samples.filterMetadata('polyID','equals',33)

### <span style="color:red"> Filter collection and add NDVI text</span>

In [93]:
# for some weird reason cloud and snow probability are missing before 2018!
res = collectionS2.\
    filterBounds(onesq).\
    filterDate('2018-03-19','2021-04-30').filterMetadata('CLOUDY_PIXEL_PERCENTAGE','less_than',30).\
    map(maskCloudAndShadows).\
    map(lambda image: ee.Image(image.normalizedDifference(['B8', 'B4']).rename('NDVI').copyProperties(image,['system:time_start','system:time_end'])))

### <span style="color:red">  Add day

In [94]:
# add day 
collection = res.map(add_dn_date_all(Year = False, days = 16))

### <span style="color:red">  Aggregate data into 16-day composites

In [96]:
tmpseas = ["%02d" % x for x in list(range(1, 23+1))]
tmpseas1 = ee.List(tmpseas)

In [118]:
# climatology 5 years (monthly composites)
seasons = ee.ImageCollection.fromImages(tmpseas1.map(aggregate)) 

In [111]:
coverage = seasons.select('NDVI').count()

In [114]:
Map.addLayer(coverage,{'min':1,'max':23,'palette':['blue','green','yellow','red']},'coverage')

In [112]:
Map

Map(bottom=12923.0, center=[38.14319750166766, -1.950408402782118], controls=(WidgetControl(options=['position…